# Chapter5-3 モデル構築
## 5-3-7 検証用スクリプトの実装

In [ ]:
!pip install azure-cognitiveservices-vision-customvision

In [ ]:
import glob
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient

In [ ]:
base_url = 'エンドポイントURL'
projectID = 'プロジェクトID'
publish_iteration_name = 'イテレーション名'
prediction_key = '予測キー'

In [ ]:
def testModel(testfiles, fishname):
    data_count = len(testfiles)
    true_count = 0
    
    # 予測用インスタンスの作成　…①
    predictor = CustomVisionPredictionClient(prediction_key, endpoint=base_url)
    
    for testfile in testfiles:
        predicts = {}
        with open(testfile, mode='rb') as f:
            # 予測実行　…②
            results = predictor.classify_image(projectID, publish_iteration_name, f.read())
        
        # 予測結果のタグの数だけループ　…③
        for prediction in results.predictions:
            # 予測した魚とその確率を紐づけて格納　…④
            predicts[prediction.tag_name] = prediction.probability
        
        # 一番確率の高い魚を予測結果として選択　…⑤
        prediction_result = max(predicts, key=predicts.get)
        
        # 予測結果が合っていれば正解数を増やす
        if fishname == prediction_result:
            true_count += 1
    
    # 正解率の算出
    accuracy = (true_count / data_count) * 100
    print('正解率:' + str(accuracy) + '%')

## 5-3-8 検証用スクリプトの実行

In [ ]:
# 検証用画像を保存したルートディレクトリパス
root_dir = 'fishimages/'
# 検証対象の魚名一覧
fishnames = ['アイゴ','オニカサゴ','カサゴ','カワハギ','キュウセンベラ',
             'クサフグ','ソウシハギ','マハゼ','マアジ','マイワシ','ミノカサゴ',
             'メジナ','メバル']

for fishname in fishnames:
    print('****' + fishname + '****')
    # testデータのリストを取得
    testfiles = glob.glob(root_dir + fishname + '/test/*')
    testModel(testfiles, fishname)